# Análisis exploratorio con datos de Netflix

En este cuaderno vamos a explorar la base de datos `netflix_titles.csv` usando **pandas** y **NumPy**.

Objetivos:
- Practicar carga y exploración de datos.
- Realizar limpieza y transformación básica.
- Responder preguntas usando agregaciones y filtros.
- Usar NumPy para calcular estadísticas sobre una variable numérica.


In [1]:
import pandas as pd
import numpy as np


## 1. Carga e inspección inicial

En esta sección cargamos el archivo `netflix_titles.csv` y revisamos su estructura general.


In [2]:
df = pd.read_csv('netflix_titles.csv')
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df.shape


(8807, 12)

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [5]:
df.rename(columns={'type': 'tipo', 'title': 'titulo','cast': 'reparto','country': 'pais','date_added': 'fecha_de_agregado','release_year': 'anio_de_lanzamiento','rating': 'valoracion','duration': 'duracion','listed_in': 'genero','description': 'descripcion'}, inplace=True)

In [6]:
df.rename(columns={'valoracion': 'clasificacion'}, inplace=True)

In [7]:
df.isna().sum()


show_id                   0
tipo                      0
titulo                    0
director               2634
reparto                 825
pais                    831
fecha_de_agregado        10
anio_de_lanzamiento       0
clasificacion             4
duracion                  3
genero                    0
descripcion               0
dtype: int64

In [8]:
df['anio_de_lanzamiento'].describe(include='all')


count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: anio_de_lanzamiento, dtype: float64

**Actividad:** Describe brevemente (en tus propias palabras) qué información contiene el dataset y qué columnas parecen más importantes para el análisis.


## 2. Limpieza y transformación básica

Vamos a:
- Convertir `date_added` a tipo fecha.
- Crear una columna `year_added`.
- Separar `duration` en número y unidad.
- Crear una columna de país principal.


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   show_id              8807 non-null   object
 1   tipo                 8807 non-null   object
 2   titulo               8807 non-null   object
 3   director             6173 non-null   object
 4   reparto              7982 non-null   object
 5   pais                 7976 non-null   object
 6   fecha_de_agregado    8797 non-null   object
 7   anio_de_lanzamiento  8807 non-null   int64 
 8   clasificacion        8803 non-null   object
 9   duracion             8804 non-null   object
 10  genero               8807 non-null   object
 11  descripcion          8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [10]:
df['fecha_de_agregado'].describe(include='all')

count                8797
unique               1767
top       January 1, 2020
freq                  109
Name: fecha_de_agregado, dtype: object

In [11]:
df['fecha_de_agregado'] = pd.to_datetime(df['fecha_de_agregado'], errors='coerce')

In [13]:
df['fecha_de_agregado'].head(20)

0    2021-09-25
1    2021-09-24
2    2021-09-24
3    2021-09-24
4    2021-09-24
5    2021-09-24
6    2021-09-24
7    2021-09-24
8    2021-09-24
9    2021-09-24
10   2021-09-24
11   2021-09-23
12   2021-09-23
13   2021-09-22
14   2021-09-22
15   2021-09-22
16   2021-09-22
17   2021-09-22
18   2021-09-22
19   2021-09-22
Name: fecha_de_agregado, dtype: datetime64[ns]

In [14]:
df['director'].fillna('Desconocido', inplace=True)
df['reparto'].fillna('Desconocido', inplace=True)
df['pais'].fillna('Desconocido', inplace=True)


C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\601319492.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['director'].fillna('Desconocido', inplace=True)
C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\601319492.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [15]:
df['fecha_de_agregado'].fillna(df['anio_de_lanzamiento'].astype(str) + "-01-01", inplace=True)

C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\2671454107.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fecha_de_agregado'].fillna(df['anio_de_lanzamiento'].astype(str) + "-01-01", inplace=True)


In [16]:
df['clasificacion'].fillna('Desconocido', inplace=True)
df['duracion'].fillna('Desconocido', inplace=True)

C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\315912804.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['clasificacion'].fillna('Desconocido', inplace=True)
C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\315912804.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [17]:
df.isna().sum()

show_id                0
tipo                   0
titulo                 0
director               0
reparto                0
pais                   0
fecha_de_agregado      0
anio_de_lanzamiento    0
clasificacion          0
duracion               0
genero                 0
descripcion            0
dtype: int64

In [18]:
netflix_peliculas = df.loc[df['tipo'] == 'Movie']
netflix_peliculas.shape


(6131, 12)

In [19]:
netflix_peliculas.to_csv('netflix_peliculas.csv', index=False)

In [20]:
netflix_peliculas = pd.read_csv('netflix_peliculas.csv')
netflix_peliculas.head(20)

,show_id,tipo,titulo,director,reparto,pais,fecha_de_agregado,anio_de_lanzamiento,clasificacion,duracion,genero,descripcion
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Desconocido,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Desconocido,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
2,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
3,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
4,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
5,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,"Klara Castanho, Lucca Picon, Júlia Gomes, Marc...",Desconocido,2021-09-22,2021,TV-PG,91 min,"Children & Family Movies, Comedies",When the clever but socially-awkward Tetê join...
6,s17,Movie,Europe's Most Dangerous Man: Otto Skorzeny in ...,"Pedro de Echave García, Pablo Azorín Williams",Desconocido,Desconocido,2021-09-22,2020,TV-MA,67 min,"Documentaries, International Movies",Declassified documents reveal the post-WWII li...
7,s19,Movie,Intrusion,Adam Salky,"Freida Pinto, Logan Marshall-Green, Robert Joh...",Desconocido,2021-09-22,2021,TV-14,94 min,Thrillers,After a deadly home invasion at a couple’s new...
8,s23,Movie,Avvai Shanmughi,K.S. Ravikumar,"Kamal Hassan, Meena, Gemini Ganesan, Heera Raj...",Desconocido,2021-09-21,1996,TV-PG,161 min,"Comedies, International Movies",Newly divorced and denied visitation rights wi...
9,s24,Movie,Go! Go! Cory Carson: Chrissy Takes the Wheel,"Alex Woo, Stanley Moore","Maisie Benson, Paul Killam, Kerry Gudjohnsen, ...",Desconocido,2021-09-21,2021,TV-Y,61 min,Children & Family Movies,From arcade games to sled days and hiccup cure...


In [21]:
netflix_series = df.loc[df['tipo'] == 'TV Show']
netflix_series.shape

(2676, 12)

In [22]:
netflix_series.to_csv('netflix_series.csv', index=False)

In [23]:
netflix_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6131 entries, 0 to 6130
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   show_id              6131 non-null   object
 1   tipo                 6131 non-null   object
 2   titulo               6131 non-null   object
 3   director             6131 non-null   object
 4   reparto              6131 non-null   object
 5   pais                 6131 non-null   object
 6   fecha_de_agregado    6131 non-null   object
 7   anio_de_lanzamiento  6131 non-null   int64 
 8   clasificacion        6131 non-null   object
 9   duracion             6131 non-null   object
 10  genero               6131 non-null   object
 11  descripcion          6131 non-null   object
dtypes: int64(1), object(11)
memory usage: 574.9+ KB


In [24]:
nxp= netflix_peliculas
nxp['director'].unique()

array(['Kirsten Johnson', 'Robert Cullen, José Luis Ucha', 'Haile Gerima',
       ..., 'Majid Al Ansari', 'Peter Hewitt', 'Mozez Singh'],
      shape=(4355,), dtype=object)

In [25]:
nxp['director'].head(20)

0                                       Kirsten Johnson
1                         Robert Cullen, José Luis Ucha
2                                          Haile Gerima
3                                        Theodore Melfi
4                                   Christian Schwochow
5                                         Bruno Garotti
6         Pedro de Echave García, Pablo Azorín Williams
7                                            Adam Salky
8                                        K.S. Ravikumar
9                               Alex Woo, Stanley Moore
10                                           S. Shankar
11                                          Rajiv Menon
12                                         Dennis Dugan
13                                        Scott Stewart
14                                       Robert Luketic
15    Ashwiny Iyer Tiwari, Abhishek Chaubey, Saket C...
16                                         Daniel Sandu
17                                       Cédric 

In [26]:
#nxp.loc[nxp['reparto'].str.contains('Vanessa Hudgens')]['titulo'].count()
nxp.loc[nxp['director'].str.contains('Robert Cullen')].head(20)

,show_id,tipo,titulo,director,reparto,pais,fecha_de_agregado,anio_de_lanzamiento,clasificacion,duracion,genero,descripcion
1,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Desconocido,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...


In [27]:
nxs = netflix_series


In [28]:
nxp['fecha_de_agregado'] = pd.to_datetime(nxp['fecha_de_agregado'], errors='coerce')

In [29]:
nxp['fecha_de_agregado'].value_counts(ascending=False)

fecha_de_agregado
2020-01-01    97
2019-11-01    75
2018-03-01    72
2019-12-31    67
2018-10-01    64
              ..
2019-06-23     1
2018-12-23     1
2019-10-26     1
2019-04-21     1
2016-05-26     1
Name: count, Length: 1533, dtype: int64

In [30]:
nxs['anio_de_lanzamiento'].value_counts(ascending=False)

anio_de_lanzamiento
2020    436
2019    397
2018    380
2021    315
2017    265
2016    244
2015    162
2014     88
2012     64
2013     63
2010     40
2011     40
2009     34
2008     23
2006     14
2007     14
2005     13
2003     10
2004      9
1999      7
2002      7
2001      5
2000      4
1998      4
1993      4
1997      4
1990      3
1992      3
1996      3
1994      2
1995      2
1986      2
1988      2
1945      1
1974      1
1991      1
1972      1
1925      1
1979      1
1977      1
1981      1
1946      1
1985      1
1967      1
1989      1
1963      1
Name: count, dtype: int64

In [31]:
nxs['fecha_de_agregado'] = pd.to_datetime(nxs['fecha_de_agregado'], errors='coerce')
nxs['fecha_de_agregado'].value_counts(ascending=False)

C:\Users\twoem\AppData\Local\Temp\ipykernel_12604\1920729059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nxs['fecha_de_agregado'] = pd.to_datetime(nxs['fecha_de_agregado'], errors='coerce')


fecha_de_agregado
2016-01-01    47
2021-07-06    31
2017-01-01    24
2017-08-01    22
2021-06-19    21
              ..
2016-11-11     1
1967-01-01     1
2021-09-05     1
2021-09-09     1
2021-09-21     1
Name: count, Length: 999, dtype: int64

In [ ]:
nxs = pd.read_csv('netflix_series.csv')
nxs.head(20)

In [ ]:
nxp['duracion'].head(20)

0      90 min
1      91 min
2     125 min
3     104 min
4     127 min
5      91 min
6      67 min
7      94 min
8     161 min
9      61 min
10    166 min
11    147 min
12    103 min
13     97 min
14    106 min
15    111 min
16    110 min
17    105 min
18     96 min
19    124 min
Name: duracion, dtype: object

In [33]:
# Separar duración en número y unidad
nxp['duracion_num'] = nxp['duracion'].str.extract(r'(\d+)').astype(float)
nxp['duracion_unit'] = nxp['duracion'].str.extract(r'([A-Za-z ]+)$')


In [34]:
nxp['duracion_num'].describe()
min_duration = nxp['duracion_num'].min()
max_duration = nxp['duracion_num'].max()
shorts = nxp.loc[nxp['duracion_num']<15]

In [35]:
shorts.to_csv('netflix_short_movies.csv', index=False)

In [ ]:
nxp['pais'].unique()

In [ ]:
# País principal (primer país de la lista)
nxp['pais_principal'] = nxp['pais'].str.split(',').str[0].str.strip()
nxp['pais_principal'].value_counts()


pais_principal
United States     2364
India              927
Desconocido        440
United Kingdom     382
Canada             187
                  ... 
Slovenia             1
Guatemala            1
Jamaica              1
Somalia              1
Croatia              1
Name: count, Length: 80, dtype: int64

In [ ]:
df.loc[df['pais'].str.contains('Colombia')]


**Actividad:** Revisa qué columnas tienen muchos valores faltantes y comenta qué decisión tomarías (eliminar filas, imputar, dejar como están) para este proyecto.


## 3. Preguntas de análisis con pandas

Responde las siguientes preguntas usando filtros, `groupby` y agregaciones.


### 3.1. Películas vs series

¿Cuántas películas y cuántas series hay en el catálogo?


In [38]:
nxp['duracion_num'].describe()

count    6128.000000
mean       99.577187
std        28.290593
min         3.000000
25%        87.000000
50%        98.000000
75%       114.000000
max       312.000000
Name: duracion_num, dtype: float64

In [37]:
df['tipo'].value_counts()


tipo
Movie      6131
TV Show    2676
Name: count, dtype: int64

In [41]:
nxs['duracion_num'] = nxs['duracion'].str.extract(r'(\d+)').astype(float)
nxs['duracion_unit'] = nxs['duracion'].str.extract(r'([A-Za-z ]+)$')

In [42]:
nxs['duracion_num'].describe()

count    2676.000000
mean        1.764948
std         1.582752
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        17.000000
Name: duracion_num, dtype: float64

In [43]:
nxs.loc[nxs['duracion_num'] == 17]

,show_id,tipo,titulo,director,reparto,pais,fecha_de_agregado,anio_de_lanzamiento,clasificacion,duracion,genero,descripcion,duracion_num,duracion_unit
204,s549,TV Show,Grey's Anatomy,Desconocido,"Ellen Pompeo, Sandra Oh, Katherine Heigl, Just...",United States,2021-07-03,2020,TV-14,17 Seasons,"Romantic TV Shows, TV Dramas",Intern (and eventual resident) Meredith Grey f...,17.0,Seasons


Si Netflix mas de 3/4 de toda su coleccion en peliculas, porque hacen series tan largas

### 3.2. Títulos añadidos por año y tipo

¿Cómo ha evolucionado el número de títulos añadidos a Netflix por año (`year_added`) y por `type`?


In [ ]:
nxp['unico_genero'] = nxp['genero'].str.split(',').str[0].str.strip()


In [ ]:
titulos_por_anio = (
    nxp.groupby(['anio_de_lanzamiento', 'unico_genero'])['show_id']
      .count()
      .reset_index(name='n_titles')
)
#titulos_por_anio.head(10)
titulos_por_anio.loc[titulos_por_anio['unico_genero'] == 'Comedies']


In [ ]:
nxp.info()

In [ ]:
nxp.groupby('pais_principal')['duracion_num'].agg(['mean', 'min', 'max', 'count']).head(10).sort_values(by='count', ascending=False)

,mean,min,max,count
pais_principal,,,,
Australia,93.393443,23.0,146.0,61
Argentina,89.607143,49.0,124.0,56
Brazil,90.517857,44.0,133.0,56
Belgium,99.444444,77.0,127.0,9
Austria,99.000000,89.0,112.0,8
Bulgaria,94.400000,87.0,110.0,5
Bangladesh,111.666667,95.0,136.0,3
Cambodia,114.500000,92.0,137.0,2
,103.000000,103.0,103.0,1


### 3.3. Duración de las películas por década

Considera solo los registros donde `type == 'Movie'`. Calcula la duración promedio por década de `release_year`.


In [ ]:
movies = df[df['type'] == 'Movie'].copy()
movies['decade'] = (movies['release_year'] // 10) * 10
duration_by_decade = (
    movies.groupby('decade')['duration_num']
          .mean()
)
duration_by_decade


### 3.4. Géneros más frecuentes

¿Cuáles son los géneros (`listed_in`) más comunes en el catálogo?


In [ ]:
genres_series = df['listed_in'].dropna().str.split(', ')
all_genres = pd.Series([g for sub in genres_series for g in sub])
top_genres = all_genres.value_counts().head(10)
top_genres


### 3.5. Países con más títulos

Usando `main_country`, ¿cuáles son los países con más títulos en el catálogo?


In [ ]:
df['main_country'].value_counts().head(10)


## 4. Uso de NumPy para estadísticas

En esta sección usamos NumPy explícitamente sobre la duración de las películas.


In [ ]:
movie_durations = movies['duration_num'].dropna().values
movie_durations[:10]


In [ ]:
mean_duration = np.mean(movie_durations)
median_duration = np.median(movie_durations)
p90_duration = np.percentile(movie_durations, 90)
mean_duration, median_duration, p90_duration


In [ ]:
long_mask = movie_durations > 120
long_ratio = long_mask.mean()
long_ratio


**Actividad:** Interpreta los resultados anteriores: ¿es común encontrar películas muy largas (> 120 minutos) en el catálogo?


## 5. Proyecto Final

Vamos a hacer una aplicacion que permita examinar la base de datos, la cual va a tener las siguientes funciones:

Buscar una series, actor o director. //
Ofrecer una serie o pelicula al azar //
Mostrar peliculas o series segun genero. //
Tiempo total de ver todo el catalogo de peliculas en Netflix //
Estadisticas por años de estreno //
Ofrecer peliculas basadas en el tiempo de duracion del usuario //



In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   show_id              8807 non-null   object        
 1   tipo                 8807 non-null   object        
 2   titulo               8807 non-null   object        
 3   director             8807 non-null   object        
 4   reparto              8807 non-null   object        
 5   pais                 8807 non-null   object        
 6   fecha_de_agregado    8807 non-null   datetime64[ns]
 7   anio_de_lanzamiento  8807 non-null   int64         
 8   clasificacion        8807 non-null   object        
 9   duracion             8807 non-null   object        
 10  genero               8807 non-null   object        
 11  descripcion          8807 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 825.8+ KB


In [46]:
df['reparto'] = df['reparto'].str.lower()
df['director'] = df['director'].str.lower()

Buscar por actor o director

In [62]:
#eleccionps=input("Ingrese pelicula o serie")

def buscar_por_actor_o_director(nombre):
    nombre = nombre.lower()
    return df.loc[(df['reparto'].str.contains(nombre, na=False)) | (df['director'].str.contains(nombre, na=False)), ['titulo', 'tipo', 'director', 'reparto']]

buscar_por_actor_o_director(input("Ingrese actor o director"))

,titulo,tipo,director,reparto


Pelicula al azar

In [84]:
def peliculas_al_azar(n):
    return df.sample(n)
peliculas_al_azar(1)


,show_id,tipo,titulo,director,reparto,pais,fecha_de_agregado,anio_de_lanzamiento,clasificacion,duracion,genero,descripcion
6532,s6533,TV Show,Cooking on High,desconocido,"josh leyva, ngaio bealum",United States,2018-06-22,2018,TV-MA,1 Season,Reality,In the first-ever competitive cannabis cooking...


In [86]:
df['genero'].replace('TV', '', regex=True, inplace=True)
df['genero'] = df['genero'].str.strip()

Busqueda por Genero

In [124]:
def buscar_por_genero(elegenero):
    genero = elegenero.lower()
    return df.loc[(df['genero'].str.contains(genero, na=False)), ['titulo', 'tipo', 'director', 'reparto']]

buscar_por_genero(input("Ingrese el genero"))

,titulo,tipo,director,reparto


Duracion  total de ver las peliculas y series en netflix

In [94]:
tiempo_total = (nxp['duracion_num'].sum() / 60)/24

print(f"{tiempo_total:.2f} dias de entretenimiento en Netflix solo con las peliculas")

423.76 dias de entretenimiento en Netflix solo con las peliculas


temporada contiene 20 capitulos, cada capitulo 60 minutos

In [98]:
tiempo_total_series = (nxs['duracion_num'].sum()*20/24)
print(f"{tiempo_total_series:.2f} dias de entretenimiento en Netflix solo con las series")

3935.83 dias de entretenimiento en Netflix solo con las series


In [111]:
duracion_estandar_series = nxs['duracion_num'].values * 1200
def estadisticas_anios(año):
    año = int(año)
    peliculas_año = nxp[nxp['anio_de_lanzamiento'] == año]
    series_año = nxs[nxs['anio_de_lanzamiento'] == año]
    duracion_peliculas = peliculas_año['duracion_num'].mean()
    minima_duracion = peliculas_año['duracion_num'].min()
    maxima_duracion = peliculas_año['duracion_num'].max()
    duracion_series = duracion_estandar_series.mean()
    minima_duracion_series = duracion_estandar_series.min()
    maxima_duracion_series = duracion_estandar_series.max()
    if año not in nxp['anio_de_lanzamiento'].values and año not in nxs['anio_de_lanzamiento'].values:
        print(f"No hay datos disponibles para el año {año}.")
        return
    else:
        print(f"En el año {año}, la duración promedio de las peliculas fue de {duracion_peliculas:.2f} minutos, la minima duración fue de {minima_duracion} minutos y la máxima duración fue de {maxima_duracion} minutos.")
        print(f"En el año {año}, la duración promedio de las series fue de {duracion_series:.2f} minutos, la minima duración fue de {minima_duracion_series} minutos y la máxima duración fue de {maxima_duracion_series} minutos.")
        print(f"En el año {año}, se lanzaron {peliculas_año.shape[0]} peliculas y {series_año.shape[0]} series.")
    return

estadisticas_anios(int(input("Ingrese el año que desea consultar")))

En el año 2020, la duración promedio de las peliculas fue de 92.14 minutos, la minima duración fue de 5.0 minutos y la máxima duración fue de 190.0 minutos.
En el año 2020, la duración promedio de las series fue de 2117.94 minutos, la minima duración fue de 1200.0 minutos y la máxima duración fue de 20400.0 minutos.
En el año 2020, se lanzaron 517 peliculas y 436 series.


In [118]:
def recomendaciones_por_duracion(duracion):
    duracion = float(duracion)
    peliculas_similares = nxp[(nxp['duracion_num'] >= duracion) & (nxp['duracion_num'] <= duracion)].head(1)
    return peliculas_similares[['titulo', 'duracion_num', 'director', 'reparto']]
recomendaciones_por_duracion(int(input("Ingrese la duracion en minutos")))

,titulo,duracion_num,director,reparto
483,From Stress to Happiness,58.0,"Alejandro De Grazia, Juan Stadler",Desconocido


In [ ]:
def menu():
    print("Bienvenido al sistema de recomendaciones de Netflix")
    print("1. Buscar por actor o director")
    print("2. Buscar por género")
    print("3. Obtener película al azar")
    print("4. Estadísticas por año")
    print("5. Recomendaciones por duración")
    print("6. Salir")
    
    while True:
        choice = input("Seleccione una opción (1-6): ")
        
        if choice == '1':
            nombre = input("Ingrese actor o director: ")
            resultados = buscar_por_actor_o_director(nombre)
            print(resultados)
        elif choice == '2':
            genero = input("Ingrese el género: ")
            resultados = buscar_por_genero(genero)
            print(resultados)
        elif choice == '3':
            azar = peliculas_al_azar(1)
            print(azar) 
        elif choice == '4':
            año = int(input("Ingrese el año que desea consultar: "))
            estadisticas_anios(año)
        elif choice == '5':
            duracion = int(input("Ingrese la duración en minutos: "))
            resultados = recomendaciones_por_duracion(duracion)
            print(resultados)
        elif choice == '6':
            print("Gracias por usar el sistema de recomendaciones de Netflix. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, seleccione una opción del 1 al 6.")


In [125]:
menu()

Bienvenido al sistema de recomendaciones de Netflix
1. Buscar por actor o director
2. Buscar por género
3. Obtener película al azar
4. Estadísticas por año
5. Recomendaciones por duración
6. Salir
Empty DataFrame
Columns: [titulo, tipo, director, reparto]
Index: []
                                             titulo     tipo         director  \
8                     The Great British Baking Show  TV Show  andy devonshire   
4461  The Great British Baking Show: The Beginnings  TV Show      desconocido   
6560                                Dangerous Roads  TV Show      desconocido   

                                                reparto  
8     mel giedroyc, sue perkins, mary berry, paul ho...  
4461  paul hollywood, mary berry, mel giedroyc, sue ...  
6560  adrian dunbar, sue perkins, charley boorman, r...  
Gracias por usar el sistema de recomendaciones de Netflix. ¡Hasta luego!


In [123]:
df.loc[df['genero'] == 'Documentaries']

,show_id,tipo,titulo,director,reparto,pais,fecha_de_agregado,anio_de_lanzamiento,clasificacion,duracion,genero,descripcion
0,s1,Movie,Dick Johnson Is Dead,kirsten johnson,desconocido,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
45,s46,Movie,My Heroes Were Cowboys,tyler greco,desconocido,Desconocido,2021-09-16,2021,PG,23 min,Documentaries,Robin Wiltshire's painful childhood was rescue...
117,s118,Movie,Final Account,luke holland,desconocido,"United Kingdom, United States",2021-09-02,2021,PG-13,94 min,Documentaries,This documentary stitches together never-befor...
221,s222,Movie,"Bob Ross: Happy Accidents, Betrayal & Greed",joshua rofé,bob ross,Desconocido,2021-08-25,2021,TV-14,93 min,Documentaries,Bob Ross brought joy to millions as the world’...
304,s305,Movie,Lady Boss: The Jackie Collins Story,laura fairrie,jackie collins,United Kingdom,2021-08-05,2021,TV-14,97 min,Documentaries,"""Queen of Sleaze"" or feminist pioneer? Powerho..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8723,s8724,Movie,What Makes a Psychopath?,rebecca harrison,desconocido,United Kingdom,2019-02-01,2017,TV-14,52 min,Documentaries,This wide-ranging documentary examines the beh...
8737,s8738,Movie,Why Are We Getting So Fat?,"milla harrison-hansley, alicky sussman",giles yeo,United Kingdom,2019-02-01,2016,TV-14,50 min,Documentaries,A Cambridge geneticist dispels misconceptions ...
8738,s8739,Movie,Why Knot,dhruv dhawan,desconocido,"Canada, India, Thailand, United States, United...",2017-10-15,2016,TV-MA,84 min,Documentaries,"Pressured by his traditional family to marry, ..."
8739,s8740,Movie,Why We Fight: The Battle of Russia,"frank capra, anatole litvak",desconocido,United States,2017-03-31,1943,TV-PG,82 min,Documentaries,This installment of Frank Capra's acclaimed do...
